### Import Data Notebook

This notebook should give me an easily accessible way of importing data from the large dataset in Snowflake to my own custom tables

In [17]:
import sys, os
import pandas as pd
import numpy as np
from dotenv import load_dotenv

sys.path.append('/Users/katringrunert/Projects/Uni/thesis/neuro-symbolic-demand-forecasting')
# when running on JupyterHub the dot-env-path needs to be adapted
load_dotenv("../.env")

datetime_format = "%Y-%m-%d %H:%M%z"

from src.neuro_symbolic_demand_forecasting.database.weather_postgres import WeatherDao
from src.neuro_symbolic_demand_forecasting.database.snowflake import SnowflakeDao


sf_dao = SnowflakeDao(
    url=os.getenv("SNOWFLAKE_URL"),
    uid=os.getenv("SNOWFLAKE_UID"),
    pwd=os.getenv("SNOWFLAKE_PASSWORD"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
)
pg_dao = WeatherDao(
    url=os.getenv("POSTGRES_HOST_WEATHER_DATA"),
    uid=os.getenv("POSTGRES_UID_WEATHER_DATA"),
    pwd=os.getenv("POSTGRES_PASSWORD_WEATHER_DATA"),
    database=os.getenv("POSTGRES_DATABASE_WEATHER_DATA"),
    schema=os.getenv("POSTGRES_SCHEMA_WEATHER_DATA"),
    port=int(os.getenv("POSTGRES_PORT_WEATHER_DATA"))
)
sf_dao.connect()
pg_dao.connect()

Connected to snowflake instance!
Connected to postgres host!


## Subsampling from ACTIVE_CONTRACT_DATA (old and big) to CONTRACT_DATA (new and smaller)

In [3]:
contract_df = sf_dao.fetch_list("SELECT * FROM SANDBOX.KATRIN.ACTIVE_CONTRACT_DATA")
contract_df

,clusterreference,degreeofisolation,hassolarpanels,solarpanelswattpeak,surfacearea,typeofhouse,yearbuilt,numberofpeople,connection_id,ean,...,contract_end_date,contract_state,pricing_type,market_segment,connection_group_id,customer_size,zip_code,city,zip_code_suffixed,last_modified
0,1068WG15,Average,False,0.0,From50,CornerHouse,From1946,Three,fa3814dc-a4eb-4153-88ba-ae580114d07b,871685900008617662,...,2024-10-17,Active,Fixed,Electricity,70dacb49-3ae3-496f-8319-ae580114d068,Small,1068,AMSTERDAM,1068 WG,2024-03-19 10:42:02
1,8426AC19-A,Average,False,0.0,From15,SemiDetached,From1975,Two,eca5959e-c22e-4658-86bd-a72901420930,871687120052650035,...,2024-04-06,Active,Fixed,Electricity,577c4d5b-fd03-4d26-8ecf-a729014208ee,Small,8426,APPELSCHA,8426 AC,2024-03-19 10:42:02
2,6718XB5-Nieuw,Good,True,0.0,From200,Detached,From2015,Three,7c0fc9ba-7638-4da8-a691-adb700d14346,871687110004168671,...,2024-04-02,Active,Fixed,Electricity,08e48d02-88d5-4a36-a358-adb700d0ed9d,Small,6718,EDE GLD,6718 XB,2024-03-19 10:42:02
3,2678ZP127,Good,True,1320.0,From100,RowHouse,From2015,Two,d6b658c4-58ad-4364-8d20-afcf00aae00f,871687800001680992,...,2024-04-05,Active,Fixed,Electricity,7ad83cb4-2a48-44b7-8ba5-afcf00aae00f,Small,2678,DE LIER,2678 ZP,2024-03-19 10:42:02
4,1181VG56,Good,True,4.0,From150,RowHouse,From1946,Three,2eaf249f-0b7c-40c4-b3ce-ac4b00ddb45a,871685900009562541,...,2024-10-30,Active,Fixed,Electricity,382fe708-2ddf-4d28-b4f6-ac4b00ddb45a,Small,1181,AMSTELVEEN,1181 VG,2024-03-19 10:42:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17029,3039SR25-B,Average,False,0.0,From50,Apartment,Until1945,Two,718a6b95-b89e-4a82-bd80-b00a00a28e17,871689290101026123,...,2024-05-23,Active,Fixed,Electricity,6044d5f2-5002-42f0-ae19-b00a00a28e17,Small,3039,ROTTERDAM,3039 SR,2024-03-19 10:44:05
17030,4531HW6,Average,False,0.0,From50,Apartment,From1975,One,c2198c6e-e0c8-4666-a9b3-ab0f014edc80,871690200022422658,...,2024-05-15,Active,Fixed,Electricity,dc18e5c9-79cf-414e-81a6-ab0f014edc80,Small,4531,TERNEUZEN,4531 HW,2024-03-19 10:44:05
17031,8913GV19,Poor,False,0.0,From15,CornerHouse,Until1945,One,108fd278-86d6-47f0-8955-abf700f2d183,871689740000025616,...,2024-09-22,Active,Fixed,Electricity,4182b295-ba28-412b-ab14-abf700f2d183,Small,8913,LEEUWARDEN,8913 GV,2024-03-19 10:44:05
17032,6351JG33,Average,True,0.0,From150,SemiDetached,From1946,Four,6f60138a-033e-4894-80f0-acdb0104569b,871688540000224191,...,2024-05-24,Active,Fixed,Electricity,96797ac0-4da2-4b01-8492-acdb0104569b,Small,6351,BOCHOLTZ,6351 JG,2024-03-19 10:44:05


In [4]:
import datetime
contract_df['runtime_total'] = (contract_df['contract_end_date'] - contract_df['contract_start_date']).apply(lambda x: x.days)
contract_df['runtime_until_now'] = contract_df['contract_start_date'].apply(lambda x: (datetime.datetime(2024,1,1,0,0) - x).days)
contract_df[['runtime_total', 'runtime_until_now', 'contract_end_date', 'contract_start_date']].head()

,runtime_total,runtime_until_now,contract_end_date,contract_start_date
0,947,657,2024-10-17,2022-03-15
1,366,270,2024-04-06,2023-04-06
2,822,730,2024-04-02,2022-01-01
3,366,271,2024-04-05,2023-04-05
4,1096,793,2024-10-30,2021-10-30


In [5]:
filtered_df = contract_df[contract_df['runtime_until_now']>730]#[['runtime_total', 'runtime_until_now', 'contract_start_date', 'contract_end_date']]
print(filtered_df['contract_start_date'].max())
filtered_df[['runtime_total', 'runtime_until_now', 'contract_start_date', 'contract_end_date']]

2021-12-31 00:00:00


,runtime_total,runtime_until_now,contract_start_date,contract_end_date
4,1096,793,2021-10-30,2024-10-30
8,1096,754,2021-12-08,2024-12-08
9,1096,883,2021-08-01,2024-08-01
10,1035,914,2021-07-01,2024-05-01
15,1096,941,2021-06-04,2024-06-04
...,...,...,...,...
17016,1096,937,2021-06-08,2024-06-08
17017,1096,944,2021-06-01,2024-06-01
17018,1096,907,2021-07-08,2024-07-08
17022,1096,822,2021-10-01,2024-10-01


In [6]:
# Samples:  15205 	 PV:  39.97  	Non-PV: 60.03
# aim to upkeep this ratio
pv=39/100.0
non_pv=1-pv

len(filtered_df[filtered_df['hassolarpanels']])/len(filtered_df), len(filtered_df[~filtered_df['hassolarpanels']])/len(filtered_df)

(0.41783635741184527, 0.5821636425881548)

In [7]:
pv_total = len(filtered_df[filtered_df['hassolarpanels']])
nonpv_totla = len(filtered_df[~filtered_df['hassolarpanels']])
total = len(filtered_df)

# pv_total, int(total*pv)
print((total-int(total*pv))/total)
print(int(total*pv))
subsampled = pd.concat([filtered_df[filtered_df['hassolarpanels']].sample(n=int(total*pv)-100), filtered_df[~filtered_df['hassolarpanels']]])
subsampled

0.6100650462170489
2278


,clusterreference,degreeofisolation,hassolarpanels,solarpanelswattpeak,surfacearea,typeofhouse,yearbuilt,numberofpeople,connection_id,ean,...,pricing_type,market_segment,connection_group_id,customer_size,zip_code,city,zip_code_suffixed,last_modified,runtime_total,runtime_until_now
3944,1115GD50,Average,True,0.0,From100,RowHouse,From1975,FiveOrMore,68ce711c-9591-4037-a559-a7c100c0514b,871685900009338870,...,Fixed,Electricity,c4d6d8b3-5e64-4378-82b3-a7c100c050d1,Small,1115,DUIVENDRECHT,1115 GD,2024-03-19 10:44:05,1096,1003
3953,6581CV14,Average,True,1650.0,From150,SemiDetached,From1965,Four,17a34041-7cf5-4dee-a24d-ac5700fe2c13,871687120053936725,...,Fixed,Electricity,c237f756-1934-4e7e-89f9-ac5700fe2c13,Small,6581,MALDEN,6581 CV,2024-03-19 10:44:05,1096,775
4622,2693EA117,Good,True,2880.0,From100,RowHouse,From2015,Four,f2f0b7b6-0166-4544-8d7b-ace500ab1448,871687800001648695,...,Fixed,Electricity,eea5f333-513d-4c14-8850-ace500ab1448,Small,2693,'S-GRAVENZANDE,2693 EA,2024-03-19 10:44:05,1096,960
4138,9651CN7,Average,True,18.0,From100,Detached,Until1945,Two,75e3c479-2b91-45d2-8e90-ac3a00f32651,871694840004711163,...,Fixed,Electricity,2c39d841-b4e0-4fb5-b3bd-ac3a00f32651,Small,9651,MEEDEN,9651 CN,2024-03-19 10:44:05,1096,832
7379,1602JB17,Good,True,3000.0,From100,CornerHouse,From1975,Two,65bfaff3-fb48-47eb-8766-a6a700e907ca,871685900004513234,...,Fixed,Electricity,0c5d50b7-7acb-4f87-9e9f-a6a700e907a9,Small,1602,ENKHUIZEN,1602 JB,2024-03-19 10:44:05,1096,765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17014,6824BG47-402,Good,False,0.0,From50,Apartment,From2015,One,dc6a400a-bee8-46cb-9be8-ad3900ef53b9,871687110003927996,...,Fixed,Electricity,3268ee71-efd9-41c4-98c8-ad3900ef53b9,Small,6824,ARNHEM,6824 BG,2024-03-19 10:44:05,1096,938
17016,3448CA5,Poor,False,0.0,From15,Apartment,From1975,One,8f779647-e29a-49e8-a017-a8d501480eca,871687400005418999,...,Fixed,Electricity,8fb15b1c-c2de-459c-b773-a8d501480eca,Small,3448,WOERDEN,3448 CA,2024-03-19 10:44:05,1096,937
17017,6462GT37,Average,False,0.0,From50,Apartment,From1975,One,e5ca7e4a-4e56-41f9-8301-ad2800ee5ec2,871688540003383000,...,Fixed,Electricity,6bdba9cf-d4cc-410a-a7e5-ad2800ee5ec2,Small,6462,KERKRADE,6462 GT,2024-03-19 10:44:05,1096,944
17018,7161RB5,Average,False,0.0,From250,Detached,Until1945,Two,c06df8e5-fea2-4357-adf4-a99300a32722,871687120055934361,...,Fixed,Electricity,0f2593f2-0286-454b-8d06-a99300a32722,Small,7161,NEEDE,7161 RB,2024-03-19 10:44:05,1096,907


In [8]:
len(subsampled[subsampled['hassolarpanels']])/len(subsampled), len(subsampled[~subsampled['hassolarpanels']])/len(subsampled)

(0.3903925434665711, 0.609607456533429)

In [30]:
df_for_insert = subsampled[['clusterreference',
 'degreeofisolation',
 'hassolarpanels',
 'solarpanelswattpeak',
 'surfacearea',
 'typeofhouse',
 'yearbuilt',
 'numberofpeople',
 'connection_id',
 'ean',
 'ean_sha256',
 'previous_contract_id',
 'contract_id',
 'initial_contract_id',
 'parent_contract_id',
 'contract_start_date',
 'contract_end_date',
 'contract_state',
 'pricing_type',
 'market_segment',
 'connection_group_id',
 'customer_size',
 'zip_code',
 'city',
 'zip_code_suffixed',
 'last_modified']]


df_for_insert['last_modified'] = datetime.datetime.now()
df_for_insert.columns = [column.upper() for column in df_for_insert.columns]

df_for_insert.reset_index(inplace=True, drop=True)
df_for_insert['CONTRACT_START_DATE'] = df_for_insert['CONTRACT_START_DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_for_insert['CONTRACT_END_DATE'] = df_for_insert['CONTRACT_END_DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_for_insert['LAST_MODIFIED'] = df_for_insert['LAST_MODIFIED'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
df_for_insert

/var/folders/ks/_jlbnqgn7ks2yq2nrk1jy21w0000gp/T/ipykernel_86463/335094463.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_insert['last_modified'] = datetime.datetime.now()


## Import Timeseries

In [4]:
from src.neuro_symbolic_demand_forecasting.helpers import constants
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector as sc

snowflake_connection = sc.connect(
    user=os.getenv("SNOWFLAKE_UID"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_URL"),
    warehouse='USER_WH',
    # region=self.snowflake_parameters.region,
    schema="KATRIN",
    database="SANDBOX",
    role="DATA_ENGINEER",
)
# write_pandas(
#     conn=snowflake_connection,
#     df=df_for_insert,
#     table_name="CONTRACT_DATA"
# )
contract_df = sf_dao.fetch_list("SELECT * FROM SANDBOX.KATRIN.CONTRACT_DATA")
contract_df

In [11]:
eans = [f"'{e.upper()}'"for e in contract_df['ean_sha256'].tolist()]
# month,year = 2, 2023
# table_name = "P4_METERREADING_WINTER"
# so far:
# 1, 2022, 2023
# 2, 2022, 2023
# 3, 2022
# 10, 2022
# 11, 2022
# 12, 2022

table_name = "P4_METERREADING_SUMMER"
# DONE
# month,year = 5, 2023
# month,year = 5, 2023
# month,year = 6, 2022
# month,year = 6, 2023
month,year = 7, 2022

# TBD 
# month,year = 7, 2023
# month,year = 8, 2023
# month,year = 8, 2022

In [12]:
i, batch_size = 0, 2000

print("For", month, year)
acc = pd.DataFrame()
while i < len(eans):
    print(i, i+batch_size)
    if i+batch_size > len(eans):
        print("Last one!")
        sub_eans = eans[i:]
    else:
        sub_eans = eans[i:i+batch_size]
    _df = sf_dao.fetch_list(f"""SELECT EAN_SHA256, LDN, ODN, READINGDATE, MEASUREUNIT, METERNUMBER
                             FROM P4.RAW.INTERVAL WHERE 
                    EAN_SHA256 IN ({','.join(sub_eans)}) 
                    AND READINGDATE LIKE '{month}/%/{year}%'
                    """)
    print("Done")
    i += batch_size
    acc = pd.concat([acc, _df])

acc

For 7 2022
0 2000
Done
2000 4000
Done
4000 6000
Last one!
Done


,ean_sha256,ldn,odn,readingdate,measureunit,meternumber
0,28748B27AD94B6E3D1C92B98A3ED1A22DE546D2DCDE19F...,13453957.000,2411113.000,7/1/2022 12:00:00 AM +02:00,5,E0054007719013020
1,BEA11819BC86FEA2B546185FD90BAC0086B39E74F5F781...,2167396.000,853843.000,7/1/2022 12:00:00 AM +02:00,5,E0054008371493921
2,BA70E201877E99246FCA6858D18FBB6E29371AFB2A49EC...,39562698.000,9817698.000,7/1/2022 12:00:00 AM +02:00,5,E0026000018774115
3,FE8180B19B478AEBF15F4C00688EA31F2C15688D95555B...,21080148.000,12219335.000,7/1/2022 12:00:00 AM +02:00,5,E0031003239459516
4,03B46BD7E6E5D728FE386C9D19DFA118CE5FF167D08FF2...,3808310.000,10523717.000,7/1/2022 12:00:00 AM +02:00,5,E0054008380528921
...,...,...,...,...,...,...
4254422,2A65128A7457D123615AE92502AD030F1AF3F861B3D835...,6161072.000,0.000,7/20/2022 3:00:00 PM +02:00,5,E0051005318875519
4254423,2A65128A7457D123615AE92502AD030F1AF3F861B3D835...,6162718.000,0.000,7/20/2022 8:30:00 PM +02:00,5,E0051005318875519
4254424,2A65128A7457D123615AE92502AD030F1AF3F861B3D835...,6161265.000,0.000,7/20/2022 3:30:00 PM +02:00,5,E0051005318875519
4254425,2A65128A7457D123615AE92502AD030F1AF3F861B3D835...,6162630.000,0.000,7/20/2022 8:15:00 PM +02:00,5,E0051005318875519


In [13]:
import datetime
acc['readingdate'] = pd.to_datetime(acc['readingdate'], format="%m/%d/%Y %I:%M:%S %p %z")
acc['readingdate'] = acc['readingdate'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S%z'))
acc['last_modified'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

acc.sort_values(by='readingdate', inplace=True)
acc['ldn'] = acc['ldn'].astype(float) 
acc['odn'] = acc['odn'].astype(float)
acc['ldn_diff'] = acc.groupby('ean_sha256')['ldn'].diff()
acc['odn_diff'] = acc.groupby('ean_sha256')['odn'].diff()

In [14]:
acc.columns = [column.upper() for column in acc.columns]
write_pandas(
    conn=snowflake_connection,
    df=acc[['EAN_SHA256','LDN','ODN','READINGDATE','METERNUMBER','LDN_DIFF','ODN_DIFF', 'LAST_MODIFIED']],
    chunk_size=200000,
    table_name=table_name
)

/var/folders/ks/_jlbnqgn7ks2yq2nrk1jy21w0000gp/T/ipykernel_35299/2150738713.py:2: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(


(True,
 76,
 15101079,
 [('sxcxlyqgsj/file10.txt',
   'LOADED',
   200000,
   200000,
   1,
   0,
   None,
   None,
   None,
   None),
  ('sxcxlyqgsj/file2.txt',
   'LOADED',
   200000,
   200000,
   1,
   0,
   None,
   None,
   None,
   None),
  ('sxcxlyqgsj/file24.txt',
   'LOADED',
   200000,
   200000,
   1,
   0,
   None,
   None,
   None,
   None),
  ('sxcxlyqgsj/file32.txt',
   'LOADED',
   200000,
   200000,
   1,
   0,
   None,
   None,
   None,
   None),
  ('sxcxlyqgsj/file40.txt',
   'LOADED',
   200000,
   200000,
   1,
   0,
   None,
   None,
   None,
   None),
  ('sxcxlyqgsj/file46.txt',
   'LOADED',
   200000,
   200000,
   1,
   0,
   None,
   None,
   None,
   None),
  ('sxcxlyqgsj/file52.txt',
   'LOADED',
   200000,
   200000,
   1,
   0,
   None,
   None,
   None,
   None),
  ('sxcxlyqgsj/file60.txt',
   'LOADED',
   200000,
   200000,
   1,
   0,
   None,
   None,
   None,
   None),
  ('sxcxlyqgsj/file7.txt',
   'LOADED',
   200000,
   200000,
   1,
   0,
   None,

In [16]:

df = sf_dao.fetch_list(f"SELECT * FROM SANDBOX.KATRIN.P4_METERREADING_SUMMER WHERE READINGDATE>='2023-05-01 00:00+02:00' AND READINGDATE<='2023-05-15 00:00+02:00'")
df.head()

KeyboardInterrupt: 